# 0. Setup Paths

In [1]:
WORKSPACE_PATH = 'Tensorflow/workspace'
SCRIPTS_PATH = 'Tensorflow/scripts'
APIMODEL_PATH = 'Tensorflow/models'
ANNOTATION_PATH = WORKSPACE_PATH+'/annotations'
# IMAGE_PATH = WORKSPACE_PATH+'/images'
IMAGE_PATH = WORKSPACE_PATH+'/images/simulator_dataset_rgb'
MODEL_PATH = WORKSPACE_PATH+'/Colab_models'
PRETRAINED_MODEL_PATH = WORKSPACE_PATH+'/pre-trained-models'

# 1. Define config file path

In [31]:
CUSTOM_MODEL_NAME = 'ssd_inception_carnd_tl'
SUB_MODEL_NAME = 'ssd_inception_v2_300'
# CUSTOM_MODEL_NAME = 'faster_rcnn_inception_carnd_tl'
# CUSTOM_MODEL_NAME = 'ssd_mobilenet_carnd_tl'
CONFIG_PATH = MODEL_PATH+'/'+CUSTOM_MODEL_NAME+'/'+SUB_MODEL_NAME +'/pipeline.config'
CONFIG_PATH

'Tensorflow/workspace/Colab_models/ssd_inception_carnd_tl/ssd_inception_v2_300/pipeline.config'

In [32]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [33]:
config = config_util.get_configs_from_pipeline_file(CONFIG_PATH)
config

{'model': ssd {
   num_classes: 4
   image_resizer {
     fixed_shape_resizer {
       height: 300
       width: 300
     }
   }
   feature_extractor {
     type: "ssd_inception_v2"
     depth_multiplier: 1.0
     min_depth: 16
     conv_hyperparams {
       regularizer {
         l2_regularizer {
           weight: 3.9999998989515007e-05
         }
       }
       initializer {
         truncated_normal_initializer {
           mean: 0.0
           stddev: 0.029999999329447746
         }
       }
       activation: RELU_6
       batch_norm {
         decay: 0.9997000098228455
         center: true
         scale: true
         epsilon: 0.0010000000474974513
         train: true
       }
     }
   }
   box_coder {
     faster_rcnn_box_coder {
       y_scale: 10.0
       x_scale: 10.0
       height_scale: 5.0
       width_scale: 5.0
     }
   }
   matcher {
     argmax_matcher {
       matched_threshold: 0.5
       unmatched_threshold: 0.5
       ignore_thresholds: false
       negative

# 3. Export trained model to an frozen graph

In [35]:
!python export_inference_graph.py --input_type image_tensor --pipeline_config_path={CONFIG_PATH} --trained_checkpoint_prefix {MODEL_PATH+'/'+CUSTOM_MODEL_NAME+'/'+SUB_MODEL_NAME + '/train_dir/model.ckpt-20000'} --output_directory={MODEL_PATH+'/'+CUSTOM_MODEL_NAME +'/'+SUB_MODEL_NAME+ '/frozen_graph'}

Converted 410 variables to const ops.


C:\Users\kaoozhi\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:469: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\kaoozhi\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:470: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\kaoozhi\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:471: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\kaoozhi\AppData\Roaming\Python\Python

# 4. Test inference

In [36]:
import os
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
import sys
from glob import glob
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

In [37]:
def load_graph(graph_file):
    """Loads a frozen inference graph"""
    graph = tf.Graph()
    with graph.as_default():
        od_graph_def = tf.GraphDef()
        with tf.gfile.GFile(graph_file, 'rb') as fid:
            serialized_graph = fid.read()
            od_graph_def.ParseFromString(serialized_graph)
            tf.import_graph_def(od_graph_def, name='')
    return graph


def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape((im_height, im_width, 3)).astype(np.uint8)

In [39]:
## Test model

PATH_TO_GRAPH = MODEL_PATH+'/'+CUSTOM_MODEL_NAME+'/'+SUB_MODEL_NAME + '/frozen_graph/frozen_inference_graph.pb' ## load SSD trained on carla simulator images
PATH_TO_LABELS = ANNOTATION_PATH + '/label_map.pbtxt'
NUM_CLASSES = 4

In [40]:
detection_graph = load_graph(PATH_TO_GRAPH)
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)
print(category_index)

{1: {'id': 1, 'name': 'Red'}, 2: {'id': 2, 'name': 'Green'}, 3: {'id': 3, 'name': 'Yellow'}, 4: {'id': 4, 'name': 'Unknown'}}


In [ ]:
FIG_SIZE = (12, 8)
# PATH_TO_IMGS = IMAGE_PATH+'/train'
# PATH_TO_IMGS = IMAGE_PATH+'/Archive/imgs'
PATH_TO_IMGS = WORKSPACE_PATH+'/images/train'
# PATH_TO_IMGS = WORKSPACE_PATH+'/images/Archive/train'
TEST_IMGS = glob(os.path.join(PATH_TO_IMGS, r'*.png'))

# random.shuffle(TEST_IMGS)

with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
        detect_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
        detect_scores = detection_graph.get_tensor_by_name('detection_scores:0')
        detect_classes = detection_graph.get_tensor_by_name('detection_classes:0')
        num_detections = detection_graph.get_tensor_by_name('num_detections:0')
        
        for idx, img_path in enumerate(TEST_IMGS):
#             random.shuffle(TEST_IMGS)
            image = Image.open(img_path)
            image_np = load_image_into_numpy_array(image)
            image_expanded = np.expand_dims(image_np, axis=0)
            
            (boxes, scores, classes, num) = sess.run(
                [detect_boxes, detect_scores, detect_classes, num_detections],
                feed_dict={image_tensor: image_expanded})
            
            print('SCORES')
            print(scores[0])
            print('CLASSES')
            print(classes[0])
            print(img_path)
            
            viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np, 
                np.squeeze(boxes),
                np.squeeze(classes).astype(np.int32),
                np.squeeze(scores),
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=5,
                line_thickness=8)
            plt.figure(figsize=FIG_SIZE)
            plt.imshow(image_np)
            plt.show()
            
